In [1]:
import dask.array as da
import dask
from dask.distributed import Client, LocalCluster, progress
from time import sleep
from dask import delayed
import numpy as np

In [2]:

cluster = LocalCluster(memory_limit='4GB', n_workers=8)
client = Client(cluster, asynchronous=True)

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39123,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:44961,Total threads: 2
Dashboard: http://127.0.0.1:35667/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:34553,


In [3]:
# Create array of random int values shape (10000,512,512) and chunks (15,512,512)

x = da.random.randint(0, 100, size=(10000, 512, 512), chunks=(15, 512, 512), dtype='int64')
x

dask.array<randint, shape=(10000, 512, 512), dtype=int64, chunksize=(15, 512, 512), chunktype=numpy.ndarray>

In [20]:
start_ends = []
def get_chunk_edges(chunk, block_info=None):
    if block_info is None:
        return None

    global start_ends
    
    start, end = block_info[0]['array-location'][0]
    print(start, end)

    return chunk

def random_cutouts(chunk, block_info=None):
    if block_info is None:
        return None
    
    sleep(1)

    start, end = block_info[0]['array-location'][0]

    for i in range(start, end):
        image = chunk[i - start]
        x_min = int(np.random.randint(0, 512, size=1, dtype='int64'))
        x_max = int(np.random.randint(x_min, 512, size=1, dtype='int64'))
        y_min = int(np.random.randint(0, 512, size=1, dtype='int64'))
        y_max = int(np.random.randint(y_min, 512, size=1, dtype='int64'))

        cutout = image[x_min:x_max, y_min:y_max]

        # Random sleep to simulate file writting

        sleep(0.0001)
        del image, cutout, x_min, x_max, y_min, y_max
    return chunk

def delayed_random_cutouts(chunk, start, end):
    sleep(10)

    for i in range(start, end):
        image = chunk[i - start]
        x_min = int(np.random.randint(0, 512, size=1, dtype='int64'))
        x_max = int(np.random.randint(x_min, 512, size=1, dtype='int64'))
        y_min = int(np.random.randint(0, 512, size=1, dtype='int64'))
        y_max = int(np.random.randint(y_min, 512, size=1, dtype='int64'))

        cutout = image[x_min:x_max, y_min:y_max]

        # Random sleep to simulate file writting

        sleep(0.0001)
        del image, cutout, x_min, x_max, y_min, y_max

In [21]:
delayed_results = []
chunk_size = x.chunksize[0]

for i, chunk in enumerate(x.partitions):
    start = i * chunk_size
    end = start + chunk_size

    result = delayed(delayed_random_cutouts)(chunk, start, end)

    delayed_results.append(result)

In [22]:
future = client.compute(delayed_results)

# See progress of computation
progress(future)

VBox()

In [13]:
for i, futur in enumerate(future):
    if futur.status != "finished":
        print(i, futur.status)


666 error


In [18]:
help(future[666])

Help on Future in module distributed.client object:

class Future(distributed.utils_comm.WrappedKey)
 |  Future(key, client=None, inform=True, state=None)
 |  
 |  A remotely running computation
 |  
 |  A Future is a local proxy to a result running on a remote worker.  A user
 |  manages future objects in the local Python process to determine what
 |  happens in the larger cluster.
 |  
 |  Parameters
 |  ----------
 |  key: str, or tuple
 |      Key of remote data to which this future refers
 |  client: Client
 |      Client that should own this future.  Defaults to _get_global_client()
 |  inform: bool
 |      Do we inform the scheduler that we need an update on this future
 |  state: FutureState
 |      The state of the future
 |  
 |  Examples
 |  --------
 |  Futures typically emerge from Client computations
 |  
 |  >>> my_future = client.submit(add, 1, 2)  # doctest: +SKIP
 |  
 |  We can track the progress and results of a future
 |  
 |  >>> my_future  # doctest: +SKIP
 |  <F

In [9]:
for y in x.partitions:
    print(y)
    break

dask.array<blocks, shape=(15, 512, 512), dtype=int64, chunksize=(15, 512, 512), chunktype=numpy.ndarray>
